# 1. Загрузка и подготовка данных

Отзывы для обучения уже загружены скриптом в SQLite3 бд.

In [2]:
import pandas as pd
import numpy as np
import sqlite3
import html
%matplotlib inline

In [12]:
data = pd.read_csv("./train_data.csv")

In [4]:
conn = sqlite3.connect('reviews_363.db')
c = conn.cursor()

In [5]:
train_data = list(c.execute("SELECT * FROM reviews;"))
conn.close()

In [6]:
data = pd.DataFrame(train_data, columns=["text", "label"])

In [7]:
data.head()

,text,label
0,"<p>Хорошая камера, получаются четкие снимки в ...",pos
1,"<p>Это мой четвертый Xiaomi, один лучше другог...",pos
2,"безрамочный, цвета оч. сочные, камера 64, откл...",pos
3,"Мощный процессор, 6 Gb памяти, отличная камера...",pos
4,"Яркий экран, отличное качество фото. Не обнару...",pos


## 1.1 Очистка.

В данные попало много лишних тегов и прочего мусора. Для начала я уберу его, а также нормализую пунктуацию.

In [8]:
def normalize_text(text):
    _t = html.unescape(text)
    _t = _t.replace("<p>"," ")
    _t = _t.replace("</p>", " ")
    _t = _t.replace("\n", " ")
    _t = _t.replace("\r", " ")
    _t = _t.replace("\t", " ")
    _t = _t.replace('"', "")
    return _t.strip()

In [9]:
def normalize_text_re(text):
    _t = text
    _t = _t.replace("   "," ")
    _t = _t.replace("  "," ")
    return _t

In [10]:
data["text"] = data["text"].apply(lambda x: normalize_text(x))
data["text"] = data["text"].apply(lambda x: normalize_text_re(x))

In [41]:
data["text"].tail(30)

10122    достоинство что это айфон)) уже. зарядка садит...
10123    Удобство, качество агрегата, простота в исполь...
10124    экран, скорость работы; внешний вид;размер; ка...
10125    Аппарат показал себя на лучшем уровне. У моего...
10126    Простота и удобство и даже тунец работает как ...
10127    У меня было 4 blackberry. Все были на BB OS. З...
10128    качественная сборка, качественное ПО. для того...
10129          8 лет,а он все также работает!. нет. супер.
10130    флеш карту принимает. удобный, лёгкий интерфей...
10131    Качество связи. Динамик (и разговорный и внешн...
10132    маленькии.уже есть опера это хорошо.аську можн...
10133    мощный приём-передача. практически всё есть. у...
10134    большие кнопки громкий звук фонарик длительная...
10135    Удобный, приятный, лёгкий. Большие клавиши и х...
10136    2 сим-карты; крупные цифры; голосовой набор; р...
10137    большой экран, цена. 2 гнезда для наушников. к...
10138    Нет люфта, клавиши нажимаются плавно. нет. Тел.

In [11]:
data.to_csv("./train_data.csv")

## 2. Векторизация

Я хочу воспользоваться библиотекой gensim, потому что я уже использовал их эмбеддинги, и они отлично показали себя в классификации, даже на плохо обработанном датасете.

Чтобы потом удобно запаковать это в sklearn pipeline, я реализую свой класс по образу `TfIdfVectorizer` из `sklearn.feature_extraction`

In [62]:
import gensim
from gensim.utils import simple_preprocess
from gensim.models import doc2vec
from tqdm import tqdm
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.feature_extraction.text import _VectorizerMixin
import numpy as np

class VectorizerTransformer(_VectorizerMixin, BaseEstimator):
    def __init__(self):
        pass

    def fit(self, raw_documents, y=None):
        X = self.preprocess(raw_documents)
        # print("Creating model...")
        model = gensim.models.doc2vec.Doc2Vec(
            vector_size=60, 
            min_count=10,
            epochs=40
        )
        # print("Building vocab...")
        model.build_vocab(X)
        # print("Training doc2vec...")
        model.train(X, total_examples=model.corpus_count, epochs=model.epochs)
        self.model = model
        return self

    def transform(self, raw_documents):
        X = self.preprocess(raw_documents)
        # print("Iinferring vectors...")
        vectorized_texts = []
        for doc_id, _ in enumerate(tqdm(X, desc="Inferring vectors: ")):
            inferred_vector = self.model.infer_vector(X[doc_id].words)
            vectorized_texts.append(inferred_vector)

        return vectorized_texts

    def preprocess(self, raw_documents):
        # print("Tokenization...")
        processed_texts = []
        for idx, text in enumerate(tqdm(raw_documents, desc="Tokenization: ")):
            processed_texts.append(doc2vec.TaggedDocument(simple_preprocess(text), [idx]))
        return processed_texts


    def fit_transform(self, texts, y=None) -> np.ndarray:
        self.fit(texts)
        X = self.transform(texts)
        
        return np.array(X)

## 3. Обучение модели 

Наши данные не сбалансированны, потому для обучения модели применю андерсемплинг (потом).

In [64]:
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_auc_score, make_scorer
from sklearn.model_selection import cross_val_score

_model = make_pipeline(VectorizerTransformer(), GradientBoostingClassifier())

cross_val_score(_model, data["text"], data.label.map({"pos": 1, "neg": 0}), n_jobs=6)

array([0.92565239, 0.92762186, 0.92807882, 0.92807882, 0.92463054])

In [61]:
_model.fit(data["text"], data.label.map({"pos": 1, "neg": 0}))

100%|██████████| 10152/10152 [00:20<00:00, 484.31it/s]


Pipeline(steps=[('vectorizertransformer', VectorizerTransformer()),
                ('gradientboostingclassifier', GradientBoostingClassifier())])

В принципе, качество уже не такое гадкое, можно попробовать сделать сабмит.

## 4. Инференс и подготовка сабмита

In [58]:
import bs4
test = []
with open("test.csv") as tfile:
    sp = bs4.BeautifulSoup(tfile)
    revs = sp.findAll("review")
    for r in revs:
        test.append(r.text)

# pd.read_csv("test.csv")

In [63]:
_model.predict(test)

100%|██████████| 100/100 [00:00<00:00, 269.89it/s]


array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

## 5. Улучшение модели

Улучшить модель можно несколькими путями:
+ 1. Улучшить препроцессинг текста
+ 2. Подобрать параметры модели векторизации
+ 3. Подобрать параметры классификатора
+ 4. Попробовать другие классификаторы

Я сделаю только п.3

## 6. Упаковка модели

In [ ]:
import pickle as pkl